In [1]:
import os
import pandas as pd
import numpy as np
from pyspark.ml import Pipeline
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext

from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import when
from pyspark.sql.functions import regexp_replace

from pyspark.ml.regression import LinearRegression
from pyspark.mllib.evaluation import RegressionMetrics

from sklearn.metrics import confusion_matrix
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.tuning import ParamGridBuilder
import numpy as np
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.regression import LinearRegression

from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, CrossValidatorModel
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.evaluation import RegressionEvaluator

In [2]:
spark = SparkSession.builder.master("local[2]").appName("Spaceship_rescue").getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(spark.sparkContext)
spark

c:\Users\jarno\AppData\Local\Programs\Python\Python310\lib\site-packages\pyspark\sql\context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [8]:
train_data = 'train.csv'
test_data = 'test.csv'
df_train = spark.read.csv(train_data, header=True, inferSchema=True)
df_test = spark.read.csv(test_data, header=True, inferSchema=True)

In [110]:
# get total number of rows
print("Total number of rows in train data: ", df_train.count())


Total number of rows in train data:  4446966


In [9]:
# kolommen die ik niet ga gebruiken
df_train = df_train.drop('Id')
df_train = df_train.drop('groupId')
df_train = df_train.drop('matchId')
df_train = df_train.drop('matchType')

df_test = df_test.drop('Id')
df_test = df_test.drop('groupId')
df_test = df_test.drop('matchId')
df_test = df_test.drop('matchType')


In [10]:
# get total number of null values in each column
print("Total number of null values in each column: ", df_train.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in df_train.columns]).show())


+-------+------+-----------+-----+-------------+-----+---------+----------+-----+-----------+-----------+-------------+--------+---------+----------+-------+------------+---------+------------+---------+---------------+------------+---------------+---------+------------+
|assists|boosts|damageDealt|DBNOs|headshotKills|heals|killPlace|killPoints|kills|killStreaks|longestKill|matchDuration|maxPlace|numGroups|rankPoints|revives|rideDistance|roadKills|swimDistance|teamKills|vehicleDestroys|walkDistance|weaponsAcquired|winPoints|winPlacePerc|
+-------+------+-----------+-----+-------------+-----+---------+----------+-----+-----------+-----------+-------------+--------+---------+----------+-------+------------+---------+------------+---------+---------------+------------+---------------+---------+------------+
|      0|     0|          0|    0|            0|    0|        0|         0|    0|          0|          0|            0|       0|        0|         0|      0|           0|        0|    

In [113]:
df_train.dtypes

[('assists', 'int'),
 ('boosts', 'int'),
 ('damageDealt', 'double'),
 ('DBNOs', 'int'),
 ('headshotKills', 'int'),
 ('heals', 'int'),
 ('killPlace', 'int'),
 ('killPoints', 'int'),
 ('kills', 'int'),
 ('killStreaks', 'int'),
 ('longestKill', 'double'),
 ('matchDuration', 'int'),
 ('maxPlace', 'int'),
 ('numGroups', 'int'),
 ('rankPoints', 'int'),
 ('revives', 'int'),
 ('rideDistance', 'double'),
 ('roadKills', 'int'),
 ('swimDistance', 'double'),
 ('teamKills', 'int'),
 ('vehicleDestroys', 'int'),
 ('walkDistance', 'double'),
 ('weaponsAcquired', 'int'),
 ('winPoints', 'int'),
 ('winPlacePerc', 'double')]

In [11]:
# drop all rows with null values
df_train = df_train.na.drop()



In [25]:
# spark ml model to predict the winPlacePerc
# https://spark.apache.org/docs/latest/ml-classification-regression.html#linear-regression

from pyspark.ml.feature import VectorAssembler
all_cols = df_train.columns
all_cols.remove('winPlacePerc')
print(all_cols)
vectorAssembler = VectorAssembler(inputCols = all_cols, outputCol = 'features')
vdf_train = vectorAssembler.transform(df_train)
vdf_train = vdf_train.select(['features', 'winPlacePerc'])
vdf_train.show(3)




['assists', 'boosts', 'damageDealt', 'DBNOs', 'headshotKills', 'heals', 'killPlace', 'killPoints', 'kills', 'killStreaks', 'longestKill', 'matchDuration', 'maxPlace', 'numGroups', 'rankPoints', 'revives', 'rideDistance', 'roadKills', 'swimDistance', 'teamKills', 'vehicleDestroys', 'walkDistance', 'weaponsAcquired', 'winPoints']
+--------------------+------------+
|            features|winPlacePerc|
+--------------------+------------+
|(24,[6,7,11,12,13...|      0.4444|
|(24,[2,6,11,12,13...|        0.64|
|(24,[0,2,6,11,12,...|      0.7755|
+--------------------+------------+
only showing top 3 rows



In [26]:
splits = vdf_train.randomSplit([0.7, 0.3])
train_df = splits[0]
test_df = splits[1]


lr = LinearRegression(featuresCol = 'features', labelCol='winPlacePerc', maxIter=10, regParam=0.3, elasticNetParam=0.8)

print("Training model... without looking for best parameters")
lr_model = lr.fit(train_df)

print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))






Training model... without looking for best parameters
Coefficients: [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.649243249044998e-06,0.0,0.0]
Intercept: 0.46500486339275177


In [ ]:



# print("Training model... looking for best parameters")

# test multiple parameters
# paramGrid = ParamGridBuilder().addGrid(lr.regParam, [0.1, 0.01]) .addGrid(lr.fitIntercept, [False, True]) .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0]).addGrid(lr.maxIter, [10, 50, 100]).build() 

# # cross validation
# crossval = CrossValidator(estimator=lr,
#                             estimatorParamMaps=paramGrid,
#                             evaluator=RegressionEvaluator(),
#                             numFolds=3)  # use 3+ folds in practice

# # Run cross-validation, and choose the best set of parameters.
# cvModel = crossval.fit(train_df)

# # Make predictions on test documents. cvModel uses the best model found (lrModel).
# predictions = cvModel.transform(test_df)

# # Evaluate best model
# evaluator = RegressionEvaluator(
#     labelCol="winPlacePerc", predictionCol="prediction", metricName="rmse")
# rmse = evaluator.evaluate(predictions)
# print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)




In [27]:
predictions = lr_model.transform(test_df)
predictions.select("prediction","winPlacePerc","features").show(5)

# Evaluate the model
evaluator = RegressionEvaluator(
    labelCol="winPlacePerc", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)


+-------------------+------------+--------------------+
|         prediction|winPlacePerc|            features|
+-------------------+------------+--------------------+
|0.46672036815100537|      0.2778|(24,[0,1,2,3,4,6,...|
|0.46500486339275177|         0.1|(24,[0,1,2,3,5,6,...|
| 0.4746063706443727|      0.6154|(24,[0,1,2,3,5,6,...|
| 0.4687191306716683|      0.2143|(24,[0,1,2,3,5,6,...|
| 0.4777381642146729|      0.6538|(24,[0,1,2,3,5,6,...|
+-------------------+------------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 0.301079


In [31]:
# predict the winPlacePerc for the test data
all_cols = df_test.columns
vectorAssembler = VectorAssembler(inputCols = all_cols, outputCol = 'features')
vdf_test = vectorAssembler.transform(df_test)
vdf_test = vdf_test.select(['features'])
vdf_test.show(3)

predictions = lr_model.transform(vdf_test)
predictions.select("prediction","features").show(5)

# save the predictions to a csv file
predictions.select("prediction").toPandas().to_csv('predictions.csv', index=True)





+--------------------+
|            features|
+--------------------+
|(24,[2,6,11,12,13...|
|[0.0,4.0,179.1,0....|
|(24,[0,2,5,6,11,1...|
+--------------------+
only showing top 3 rows

+-------------------+--------------------+
|         prediction|            features|
+-------------------+--------------------+
| 0.4689146184231902|(24,[2,6,11,12,13...|
|0.47841638702607553|[0.0,4.0,179.1,0....|
| 0.4702431372243494|(24,[0,2,5,6,11,1...|
| 0.4770532921600213|(24,[2,6,11,12,13...|
| 0.4847065711396721|[0.0,4.0,330.2,1....|
+-------------------+--------------------+
only showing top 5 rows

